In [ ]:
---
title: Signal Denoising
description: Remove noise from a sinusoidal signal
author: Daning H.
show-code: False
show-prompt: False
params:
    ns:
        input: numeric
        label: Level of noise
        value: 1.0
        step: 0.1
---

Suppose we have a sinusoidal signal (could be, e.g., from the IMU measurement of a drone)
$$
f(t) = \sin(2\pi t)
$$
The measurement of this signal is contaminated by some noise $\epsilon(t)$
$$
y(t) = f(t) + \epsilon(t)
$$
We wish to use **Fourier transform** (FT) to reduce such noise and recover the original signal.

From FT, we can obtain the spectrum of the measure signal, i.e., the magnitudes of the frequency components in the signal.  Looking at the example below, one can see that there is a spike at frequency 1 Hz, which corresponds to our original sinusoidal signal.  But there are also some low-amplitude high-frequency components throughout the spectrum; these are due to the noise.

Intuitively, we could **filter** the spectrum by removing the high-frequency components, and perform **inverse FT** to recover time-domain signal from the filtered spectrum.  This technique is called **low-pass filtering** ("low" for low-frequency).  From the results below, one can see that the filtering can effectively remove the noise and recover the sinusoidal signal.  Yet, for this particular example, there is an even better way to denoise the signal - think about what it is.

In [1]:
ns = 1.0

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

ts = np.arange(2,21)*0.5
Nt = len(ts)

# Parameter setup
T = 5.0         # Sample Period
fs = 100.0      # sample rate, Hz
n = int(T * fs) # total number of samples
t = np.linspace(0, T, n+1)
# sin wave
sig = 1.5*np.sin(2*np.pi*t)
noise = np.random.randn(n+1)  # Random noise, more likely in real life

# Prepare data
data = sig + ns*noise
spec = np.fft.fft(data)
freq = np.fft.fftfreq(n+1, d=1/fs)
def getSpec(fr, sp):
    _s = np.sqrt(sp.real**2+sp.imag**2)/(n+1)
    _m = fr>=0
    return fr[_m], _s[_m]*2
fr, sp = getSpec(freq, spec)

splp = np.zeros((Nt, len(fr)))
filt = np.zeros((Nt, n+1))
for i in range(Nt):
    mask = np.abs(freq) > ts[i]
    _sp = np.copy(spec)
    _sp[mask] = 0.0
    _, splp[i] = getSpec(freq, _sp)
    filt[i] = np.fft.ifft(_sp).real

# Make the plots
fig = make_subplots(rows=2, cols=2,
                    vertical_spacing=0.1, horizontal_spacing=0.05,
                    subplot_titles=("Raw Signal", "Noisy Spectrum", "Filtered Signal", "Filtered Spectrum"))

# Add traces, one for each slider step
## Varying signals
### Filtered signal
for _i in range(Nt):
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="green", width=2, dash='dash'), mode='lines',
            name="Filtered signal", x=t, y=filt[_i]), row=2, col=1)
### Filtered spectrum
for _i in range(Nt):
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="black", width=2),
            name="Filtered spectrum", x=fr, y=splp[_i], showlegend=False), row=2, col=2)
fig.data[Nt-1].visible = True
fig.data[2*Nt-1].visible = True
## Fixed signals
### Original signals
fig.add_trace(go.Scatter(
    visible=True, line=dict(color="red", width=1, dash='dash'), mode='lines',
    name="Noisy signal", x=t, y=data), row=1, col=1)
fig.add_trace(go.Scatter(
    visible=True, line=dict(color="blue", width=2), mode='lines',
    name="Clean signal", x=t, y=sig), row=1, col=1)
### Reference spectrum
fig.add_trace(go.Scatter(
    visible=True, line=dict(color="black", width=2),
    name="Noisy spectrum", x=fr, y=sp, showlegend=False), row=1, col=2)
### Reference signal
fig.add_trace(go.Scatter(
    visible=True, line=dict(color="blue", width=2), mode='lines',
    name="Clean signal", x=t, y=sig, showlegend=False), row=2, col=1)

# Create and add slider
steps = []
for i in range(Nt):
    step = dict(
        method="update",
        args=[{"visible": [False] * (2*Nt) + [True]*4}],
        label=f"{ts[i]} Hz"
    )
    step["args"][0]["visible"][i] = True
    step["args"][0]["visible"][i+Nt] = True
    steps.append(step)

sliders = [dict(
    active=Nt-1,
    currentvalue={"prefix": "Threshold = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    autosize=False,
    width=1000,
    height=600)
fig.update_yaxes(row=1, col=1, range=[-3,3])
fig.update_xaxes(row=1, col=2, range=[0,12])
fig.update_xaxes(row=2, col=1, title_text='time, s')
fig.update_yaxes(row=2, col=1, range=[-3,3])
fig.update_xaxes(row=2, col=2, range=[0,12], title_text='Frequency, Hz')

fig.show()